In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload  
%load_ext autoreload
%autoreload 2

In [2]:
# Autoreload
import sys
from typing import List

sys.path.append('../')

import numpy as np
from IPython.display import Audio, display


import nltk
import torch
import numpy as np
import datasets
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM

In [3]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

from llmsearch.utils.mem_utils import gc_cuda

print(f"Device - {device}")

def beep(duration = 1, frequency=440, rhythm=1):
    sample_rate = 44100  # Standard audio sample rate
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    audio_data = np.sin(2*np.pi*frequency*t)  # Generate a sine wave
    audio_data *= np.where(np.arange(len(audio_data)) % rhythm == 0, 1, 0)  # Apply rhythm
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

Device - mps


In [4]:
dataset = datasets.load_dataset("samsum")

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
sample_size = 100
samples_to_tune_on = datasets.Dataset.from_dict(dataset["train"][:sample_size])
samples_to_tune_on = samples_to_tune_on.rename_columns(column_mapping = {'dialogue' : 'X', 'summary' : "y"})

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = False)
model =  AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)



In [7]:
import langchain

X = samples_to_tune_on[0]['X']

pt = langchain.PromptTemplate.from_template("Conversation: {X}\nSummary:")

pt.format_prompt(X = X).to_string()

"Conversation: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\nSummary:"

In [8]:
import evaluate

rouge_metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)

    return result['rouge2']

In [9]:
from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import get_total_available_ram, get_gpu_information

from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

seed = 42

total_available_ram_memory = get_total_available_ram()

print(f"Total available ram memory - {total_available_ram_memory}\n")

tuner_ob = Tuner(model = "model",tokenizer = tokenizer,dataset = samples_to_tune_on,device = device, batch_size = 512,tokenizer_encoding_kwargs={'padding': True, 'truncation': True, 'max_length': 512},tokenizer_decoding_kwargs = {'skip_special_tokens' : True,  'spaces_between_special_tokens' : False}, scorer = get_rouge_score, prompt_template = pt, is_encoder_decoder = True, seed = seed)


Total available ram memory - 6



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
# Earlier
from llmsearch.utils.model_utils import seed_everything


"""
parameters and how they affect do_sample == False
1. temperature - output does not change - greedy decoding
2. top_k - output does not change - greedy decoding
3. repetition_penalty - output changes
4. no_repeat_ngram_size - output changes
"""

# seed_everything(seed)

initial_generation_params1 = {
    'max_new_tokens' : 120,
}
# score, outputs1 = tuner_ob.get_score(initial_generation_params1)

In [11]:
hyp_param_grid = {
    "max_new_tokens": [120],
    "temperature": list(np.linspace(start=0.1, stop=1.0,num=10)),
    'top_k' : list(map(int,np.linspace(start=10, stop=50,num=5))),
    "top_p": [0.75, 0.8, 0.9, 1.0],
    'do_sample' : [True, False],
    'generation_seed' : [42],
    'repetition_penalty' : [1.0, 1.2],
    'no_repeat_ngram_size' : [0,2,3],
}

scorer = make_scorer(score_func=get_rouge_score, greater_is_better=True)


clf = RandomizedSearchCV(
    estimator=tuner_ob.estimator,
    param_distributions=hyp_param_grid,
    n_iter = 1,
    scoring=scorer,
    cv=5,
    random_state = 42,
    n_jobs=1,
    verbose=3,
)

"""
5 fold means, whole sample set of 100 examples will be split into 80:20 ratio
for each hyper_parameter set we have a model f(hyper_params)
    - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)
    - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold
"""


'\n5 fold means, whole sample set of 100 examples will be split into 80:20 ratio\nfor each hyper_parameter set we have a model f(hyper_params)\n    - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)\n    - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold\n'

In [12]:
import inspect
inspect.isclass()

TypeError: isclass() missing 1 required positional argument: 'object'

In [13]:
len(tuner_ob.dataset['X'])

100

In [14]:
def foo(a):
    for item in a:
        print(f"input to function is {a}")
        foo()

In [16]:
clf.fit(X=tuner_ob.dataset["X"], y=tuner_ob.dataset["y"])

calling overriden clone
Fitting 5 folds for each of 1 candidates, totalling 5 fits
calling overriden clone
Cloning - top_p 0.75
in native - 0.75
Cloning - top_k 10
in native - 10
Cloning - temperature 0.4
in native - 0.4
Cloning - repetition_penalty 1.0
in native - 1.0
Cloning - no_repeat_ngram_size 3
in native - 3
Cloning - max_new_tokens 120
in native - 120
Cloning - generation_seed 42
in native - 42
Cloning - do_sample True
in native - True
Performing inference with batch_size - 512


  0%|          | 0/1 [00:00<?, ?it/s]

[CV 1/5] END do_sample=True, generation_seed=42, max_new_tokens=120, no_repeat_ngram_size=3, repetition_penalty=1.0, temperature=0.4, top_k=10, top_p=0.75;, score=nan total time=   0.1s
calling overriden clone
Cloning - top_p 0.75
in native - 0.75
Cloning - top_k 10
in native - 10
Cloning - temperature 0.4
in native - 0.4
Cloning - repetition_penalty 1.0
in native - 1.0
Cloning - no_repeat_ngram_size 3
in native - 3
Cloning - max_new_tokens 120
in native - 120
Cloning - generation_seed 42
in native - 42
Cloning - do_sample True
in native - True
Performing inference with batch_size - 512


/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:825: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 814, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/pr

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[CV 4/5] END do_sample=True, generation_seed=42, max_new_tokens=120, no_repeat_ngram_size=3, repetition_penalty=1.0, temperature=0.4, top_k=10, top_p=0.75;, score=nan total time=   0.1s
calling overriden clone
Cloning - top_p 0.75
in native - 0.75
Cloning - top_k 10
in native - 10
Cloning - temperature 0.4
in native - 0.4
Cloning - repetition_penalty 1.0
in native - 1.0
Cloning - no_repeat_ngram_size 3
in native - 3
Cloning - max_new_tokens 120
in native - 120
Cloning - generation_seed 42
in native - 42
Cloning - do_sample True
in native - True
Performing inference with batch_size - 512


  0%|          | 0/1 [00:00<?, ?it/s]

[CV 5/5] END do_sample=True, generation_seed=42, max_new_tokens=120, no_repeat_ngram_size=3, repetition_penalty=1.0, temperature=0.4, top_k=10, top_p=0.75;, score=nan total time=   0.1s
calling overriden clone
calling overriden clone


RandomizedSearchCV(cv=5,
                   estimator=EstimatorWrapper(batch_size=512, device='mps',
                                              disable_batch_size_cache=False,
                                              do_sample=True,
                                              generation_seed=42,
                                              is_encoder_decoder=True,
                                              is_fitted_=True,
                                              max_new_tokens=120, model='model',
                                              no_repeat_ngram_size=3,
                                              repetition_penalty=1.0,
                                              scorer=make_scorer(get_rouge_score),
                                              temperature=0.4,
                                              tokenizer=T5Tokenizer(name_or_path=...
                   param_distributions={'do_sample': [True, False],
                                        'generation_seed': [42],
                                        'max_new_tokens': [120],
                                        'no_repeat_ngram_size': [0, 2, 3],
                                        'repetition_penalty': [1.0, 1.2],
                                        'temperature': [0.1, 0.2,
                                                        0.30000000000000004,
                                                        0.4, 0.5, 0.6,
                                                        0.7000000000000001, 0.8,
                                                        0.9, 1.0],
                                        'top_k': [10, 20, 30, 40, 50],
                                        'top_p': [0.75, 0.8, 0.9, 1.0]},
                   random_state=42, scoring=make_scorer(get_rouge_score),
                   verbose=3)

In [ ]:
clf.best_params_

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.best_estimator_.set_params(**clf.best_params_).get_params()


In [ ]:
clf.fi

In [ ]:
from sklearn.base import clone

clone(1)

In [ ]:
dir(clf)

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_estimator_.get_params()